<a href="https://colab.research.google.com/github/Hinna0818/DL-self-practice/blob/main/pytorch/6optimize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## prerequisite code
# 导入库、准备模型和输入
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

100%|██████████| 26.4M/26.4M [00:01<00:00, 16.2MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 306kB/s]
100%|██████████| 4.42M/4.42M [00:00<00:00, 5.60MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 7.84MB/s]


In [2]:
## 超参数
# 学习率
learning_rate = 1e-3

# 批量大小（64个样本一起作为一个batch来训练）
batch_size = 64

# 对整个数据集循环训练5次
epochs = 5

In [4]:
## loss function
loss_fn = nn.CrossEntropyLoss()

In [5]:
## optimizer优化器
# 这里使用随机梯度下降SGD为例
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

In [6]:
## 完整的训练和测试实例
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.297624  [   64/60000]
loss: 2.291384  [ 6464/60000]
loss: 2.271060  [12864/60000]
loss: 2.260794  [19264/60000]
loss: 2.242764  [25664/60000]
loss: 2.209429  [32064/60000]
loss: 2.223291  [38464/60000]
loss: 2.183870  [44864/60000]
loss: 2.191215  [51264/60000]
loss: 2.144919  [57664/60000]
Test Error: 
 Accuracy: 45.4%, Avg loss: 2.146839 

Epoch 2
-------------------------------
loss: 2.162864  [   64/60000]
loss: 2.152057  [ 6464/60000]
loss: 2.092052  [12864/60000]
loss: 2.108249  [19264/60000]
loss: 2.046626  [25664/60000]
loss: 1.986766  [32064/60000]
loss: 2.018386  [38464/60000]
loss: 1.929486  [44864/60000]
loss: 1.953790  [51264/60000]
loss: 1.862685  [57664/60000]
Test Error: 
 Accuracy: 56.5%, Avg loss: 1.868577 

Epoch 3
-------------------------------
loss: 1.906936  [   64/60000]
loss: 1.873723  [ 6464/60000]
loss: 1.752507  [12864/60000]
loss: 1.797558  [19264/60000]
loss: 1.681628  [25664/60000]
loss: 1.633149  [32064/600